In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
#q1
q = bq.query(
"""
SELECT geo_id 
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
""")
result = q.to_dataframe()
print(result['geo_id'][0])

55025


In [3]:
#q2
q = bq.query(
"""
SELECT state_fips_code, COUNT(*) as county_count
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY county_count DESC
LIMIT 5
""")
result = q.to_dataframe()
result.set_index('state_fips_code')['county_count'].to_dict()

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [4]:
#q3
def calculate_cost(query, job_config):
    query_job = bq.query(query, job_config=job_config)
    bytes_billed = query_job.total_bytes_billed
    cost = (bytes_billed / (1024 ** 4)) * 5  # 비용 계산 (단위: 달러)
    return cost

# setting for not using cache
job_config = bigquery.QueryJobConfig(use_query_cache=False)

# first query
q1 = """
SELECT geo_id 
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
"""
cost_q1 = calculate_cost(q1, job_config)

# second query
q2 = """
SELECT state_fips_code, COUNT(*) as county_count
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY county_count DESC
LIMIT 5
"""
cost_q2 = calculate_cost(q2, job_config)

costs = {'q1': cost_q1, 'q2': cost_q2}
print(costs)

{'q1': 4.76837158203125e-05, 'q2': 4.76837158203125e-05}


In [5]:
ds = bigquery.Dataset("cs544-f23-398504.p8")
ds

Dataset(DatasetReference('cs544-f23-398504', 'p8'))

In [6]:
bq.create_dataset(ds, exists_ok=True)

Dataset(DatasetReference('cs544-f23-398504', 'p8'))

In [7]:
uri = "gs://junhojunho/hdma-wi-2021.parquet"

table_id = "p8.hdma"

job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET)
load_job = bq.load_table_from_uri(uri, table_id, job_config=job_config)

load_job.result()

LoadJob<project=cs544-f23-398504, location=US, id=afbf5388-05e0-45c0-98c5-65324d45568d>

In [8]:
#q4
datasets = [ds.dataset_id for ds in bq.list_datasets("cs544-f23-398504")]
print(datasets)

['p8']


In [9]:
#q5
query = """
SELECT c.county_name, COUNT(*) as loan_count
FROM `p8.hdma` as h
JOIN `bigquery-public-data.geo_us_boundaries.counties` as c
ON h.county_code = c.geo_id
GROUP BY c.county_name
ORDER BY loan_count DESC
LIMIT 10
"""

query_job = bq.query(query)

# convert the result to dictionary
result = query_job.to_dataframe()
loan_counts = result.set_index('county_name')['loan_count'].to_dict()

print(loan_counts)


{'Milwaukee': 93140, 'Dane': 77114, 'Waukesha': 68318, 'Brown': 31230, 'Racine': 26014, 'Outagamie': 23046, 'Kenosha': 21488, 'Washington': 21452, 'Rock': 19668, 'Winnebago': 18620}


In [10]:
# google sheet 287

In [11]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table_id = "cs544-f23-398504.p8.applications"
table = bigquery.Table(table_id)
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [12]:
#q6
# chosen income
chosen_income = 250000

# BigQuery
query = f"""
SELECT COUNT(*) as application_count
FROM `p8.applications`
WHERE income = {chosen_income}
"""

query_job = bq.query(query)

result = query_job.to_dataframe()
application_count = result['application_count'][0]

print(application_count)

9


In [13]:
#q7
query = """
SELECT c.county_name, COUNT(*) as application_count
FROM `p8.applications` as a
JOIN `bigquery-public-data.geo_us_boundaries.counties` as c
ON ST_CONTAINS(c.county_geom, ST_GEOGPOINT(a.longitude, a.latitude))
WHERE c.state_fips_code = '55'
GROUP BY c.county_name
ORDER BY application_count DESC
LIMIT 10
"""

query_job = bq.query(query)

result = query_job.to_dataframe()
application_counts = result.set_index('county_name')['application_count'].to_dict()

print(application_counts)


{'Dane': 79, 'Door': 21, 'Brown': 7, 'Sheboygan': 7, 'Columbia': 5, 'Bayfield': 5, 'Oneida': 4, 'Milwaukee': 4, 'Walworth': 4, 'Kewaunee': 3}


In [14]:
%load_ext google.cloud.bigquery

In [15]:
%%bigquery
CREATE OR REPLACE MODEL `cs544-f23-398504.p8.new_model`
OPTIONS(model_type='LINEAR_REG', input_label_cols=['loan_amount']) AS
SELECT
  loan_amount,
  income,
  loan_term
FROM
  `cs544-f23-398504.p8.hdma`

Query is running:   0%|          |

""


In [18]:
#q8
import time
model_name = 'new_model' 
while True:
    models = bq.list_models('cs544-f23-398504.p8')
    if model_name in [model.model_id for model in models]:
        break
    time.sleep(5)

query = f"""
SELECT r2_score
FROM ML.EVALUATE(MODEL `p8.{model_name}`)
"""

query_job = bq.query(query)

r2_score = query_job.to_dataframe()['r2_score'][0]
print(r2_score)

0.29377696427746547


In [19]:
#q9
query = f"""
SELECT processed_input, weight
FROM ML.WEIGHTS(MODEL `p8.{model_name}`)
WHERE processed_input = 'income'
"""

query_job = bq.query(query)
df = query_job.to_dataframe()

print(df)

  processed_input    weight
0          income  0.806046


In [23]:
#q10
query = """
SELECT income, loan_amount
FROM `cs544-f23-398504.p8.applications`
"""
query_job = bq.query(query)
df = query_job.to_dataframe()

# Add a column for loan_term
df['loan_term'] = 360

query = f"""
SELECT predicted_loan_amount
FROM ML.PREDICT(MODEL `p8.{model_name}`, 
                (SELECT income, 360 AS loan_term FROM `cs544-f23-398504.p8.applications`))
"""
query_job = bq.query(query)
predicted = query_job.to_dataframe()['predicted_loan_amount']

ratio = (df['loan_amount'] > predicted).mean()

print(ratio)

0.597972972972973
